# Семинар 10: продвинутые функции

### Рекурсия

Пусть мы хотим написать функцию быстрого возведения числа в степень:

```
def pow(base: Union[int, float], deg: int)
```

Известно, что:
```
pow(a, n)
    = pow(a, n // 2) ** 2, если n -- четное
    = a * pow(a, n // 2) ** 2, если n -- нечетное
```

In [ ]:
def pow(base: int | float, deg: int) -> int | float:
    if base == 0:
        return 0  # а без этого получим DivisionByZeroError, если deg < 0 [1]

    if deg == 0:
        return 1  # без этого получим бесконечное выполнение
    elif deg < 0:
        deg *= -1
        base = 1 / base  # [1] вот тут

    if not deg % 2:
        return pow(base, deg // 2) ** 2
    return base * pow(base, deg // 2) ** 2


In [ ]:
pow(2, 2)

Что тут произошло? Правильно, мы вызвали из функции саму себя, получается так называемая рекурсия.

С рекурсией стоит быть аккуратным: неаккуратное ее использование приведет к бесконечной работе программы, а также она выжрет у вас всю память компа или питон прибьет ее раньше.

Под капотом рекурсия работает как стэк вызовов. Каждый новый вызов помещается на его верхушку, после выполнения работы функции на этапе рекурсии этот вызов со стэка снимается. Проще всего это заметить если Сделать искусственный пример, где выдадим ошибку в какой-то момент времени.

In [ ]:
def recursive(n: int):
    if n == 0:
        raise RuntimeError("Just checking call stack")

    recursive(n - 1)


In [ ]:
recursive(6)

### Проблема лимита рекурсии

Пусть мы хотим рекурсивно сумму для всех чисел от 1 до n (примечание: ДА, я знаю, что это можно просто сделать циклом, это игрушечный пример)

In [ ]:
def recursive(n: int):
    if n == 1:
        return 1

    return n + recursive(n - 1)

In [ ]:
recursive(3)

In [ ]:
recursive(10000)  # ой, а че это мы такое поймали?

In [ ]:
import sys

sys.getrecursionlimit()  # по умолчанию в питоне очень маленький лимит на рекурсивные вызовы

In [ ]:
sys.setrecursionlimit(10 ** 6)  # надо быть аккуратным

recursive(10000)

### Проблема отсутствия мемоизации

Пусть мы хотим посчитать $n$-ое число Фибоначчи

$F_1 = F_2 = 1$

$F_n = F_{n - 1} + F_{n - 2}$

In [ ]:
def fib(n: int) -> int:
    if n <= 2:
        return 1
    return fib(n - 1) + fib(n - 2)  # рекурсивно берем предыдущие числа Фибоначчи

А теперь давайте подумаем: сколько будет работать такой код?

In [ ]:
%%time
fib(8)

In [ ]:
%%time
fib(34)  # тут уже будет долго

In [ ]:
%%time
fib(40)  # и чем дальше, тем хуже, а 40 -- это же еще не то чтобы много

Проблема заключается в том, что у нас есть дубликаты вызовов. Поэтому питон делает много дополнительных действий. Чем дальше по номеру число Фибоначчи -- тем больше придется решать одинаковых абсолютно вызовов рекурсии. Решить это можно мемоизацией:

In [ ]:
from collections import defaultdict


SENTINEL = object()


fib_cache = defaultdict(lambda: SENTINEL)


def set_cache(n: int, result: int):
    fib_cache[n] = result


def fib(n: int) -> int:
    if fib_cache[n] is not SENTINEL:
        return fib_cache[n]

    if n <= 2:
        set_cache(n, 1)
        return 1

    result = fib(n - 1) + fib(n - 2)
    set_cache(n, result)
    return result

In [ ]:
%%time
fib(1000)  # быстро? быстро!

Очевидно, что это уже получается громоздко и неудобно, поэтому кэширование в питоне есть в том числе и встроенное.

Есть так называемые декораторы `@cache` и `@lru_cache` (least recently used). Второй выкидывает из кэша элементы, когда его размер достиг заданного максимума. При этом `@cache` устроен вот так:

```(python)
def cache(user_function, /):
    return lru_cache(maxsize=None)(user_function)
```

У `@lru_cache` параметр maxsize по умолчанию равен 128, можно увеличить при желании.

In [ ]:
from functools import lru_cache

# lru_cache -- можно еще указать параметр maxsize
@lru_cache  # <--- это декоратор, про них поговорим чуть позже
def fib(n: int) -> int:
    if n <= 2:
        return 1

    return fib(n - 1) + fib(n - 2)

In [ ]:
%%time
fib(1000)